In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

  Cloning https://github.com/terrierteam/pyterrier_t5.git to /tmp/pip-req-build-zs1qm695
  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_t5.git /tmp/pip-req-build-zs1qm695
  Resolved https://github.com/terrierteam/pyterrier_t5.git to commit 77a052046ab784da9000d6f8c8d0f1b823036314
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

In [ ]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
import json

#Load index
index = pt.IndexFactory.of('./drive/MyDrive/information-retrieval/index_text')

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started (triggered by IndexFactory.of) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


20:51:34.852 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.2 GiB of memory would be required.


In [ ]:
a = pt.text.get_text(index, 'text')

In [ ]:
monoT5 = MonoT5ReRanker() # loads castorini/monot5-base-msmarco by default
duoT5 = DuoT5ReRanker() # loads castorini/duot5-base-msmarco by default

bm25 = pt.terrier.Retriever(index, wmodel="BM25")

mono_pipeline = pt.rewrite.tokenise() >> bm25 >> pt.text.get_text(index, "text") >> monoT5
duo_pipeline = mono_pipeline % 100 >> duoT5 # apply a rank cutoff of 5 from monoT5 since duoT5 is too costly to run over the full result list

In [ ]:
duo_pipeline.search("information retrieval")

monoT5:   0%|          | 0/250 [00:00<?, ?batches/s]

duoT5:   0%|          | 0/1 [00:00<?, ?queries/s]

/usr/local/lib/python3.11/dist-packages/pyterrier_t5/__init__.py:145: UserWarning: A large number of results per query was detected (100). Since DuoT5 is an O(n^2) operation, this will take a considerable amount of time to process. Consider first reducing the size of the results using the % operator.
  warnings.warn(f'A large number of results per query was detected ({len(group)}). Since DuoT5 '


,qid,docid,docno,query_0,query,text,score,rank
20,1,4304935,1958488,information retrieval,information retrieval,Information retrieval \n Information retrieval...,931.807141,0
5,1,4361900,2417796,information retrieval,information retrieval,Legal information retrieval \n Legal informati...,655.957553,1
56,1,2693643,2924469,information retrieval,information retrieval,Multimedia Information Retrieval \n Multimedia...,653.154591,2
3,1,613155,0249777,information retrieval,information retrieval,Adversarial information retrieval \n Adversari...,640.110693,3
29,1,4567551,4052862,information retrieval,information retrieval,Temporal information retrieval \n Temporal Inf...,634.463613,4
...,...,...,...,...,...,...,...,...
74,1,4304926,1958416,information retrieval,information retrieval,Information centre \n An information centre is...,-482.281117,95
99,1,3802066,2588252,information retrieval,information retrieval,Lorraine Borman \n Lorraine Borman is an Ameri...,-483.349262,96
44,1,2174406,3426211,information retrieval,information retrieval,Queries per second \n Queries Per Second (QPS)...,-500.742822,97
73,1,3146890,1958414,information retrieval,information retrieval,Information capture \n Information capture is ...,-524.385921,98


In [ ]:
import pandas as pd
qrels = pd.read_csv('./queries/train_qrels.csv',
                            dtype={'qid': 'object', 'docno': 'object', 'label': 'int64'})

topics = pd.read_csv('./queries/train_queries.csv',
                      dtype={'qid': 'object', 'query': 'object'})

ans = pt.Experiment([mono_pipeline], topics, qrels,
                            eval_metrics=['recip_rank', 'ndcg_cut', 'recall'],
                            names=['mono'])

monoT5:   0%|          | 0/57806 [00:00<?, ?batches/s]

In [ ]:
ans

,name,recip_rank,nDCG@5,nDCG@10,nDCG@15,nDCG@20,nDCG@30,nDCG@100,nDCG@200,nDCG@500,nDCG@1000,R@5,R@10,R@15,R@20,R@30,R@100,R@200,R@500,R@1000
0,mono,0.747591,0.465313,0.462807,0.465031,0.464472,0.47252,0.522244,0.544394,0.561796,0.566488,0.215017,0.297576,0.359906,0.39677,0.457297,0.611678,0.673453,0.73098,0.750237


In [ ]:
queries = pd.read_csv('./queries/test_queries.csv')

#Transform applies the search, cutoff at 100
df_ans = (mono_pipeline % 100).transform(queries)
df_ans = df_ans.sort_values(['qid', 'rank']) #Sort by rank, just in case

#Detailed result
df_ans.to_csv(f'./queries/t5-test-results.csv', index=False)

#Formatted result, for submitting to kaggle
( df_ans[['qid', 'docno']]
    .rename(columns={'qid': 'QueryId', 'docno': 'EntityId'})
    .to_csv(f'./queries/t5-submit-results.csv', index=False)
)

monoT5:   0%|          | 0/57708 [00:00<?, ?batches/s]